In [0]:
from google.colab import drive

#drive.mount('/content/drive')

In [0]:
import sys
sys.path.append('/content/drive/reco/')
#from content.drive.reco.data.start import start
import pandas as pd
data = pd.read_csv('/path/to/ratings.csv', header=0)



In [0]:
#data.iloc[3]
data = data[data["rating"]>=4]


Remove users with less than 5 interactions

In [0]:
movie_dist = data[["movieId"]].groupby("movieId",as_index=False).size()
user_dist = data[["userId"]].groupby("userId",as_index=False).size()
data = data[data['userId'].isin(user_dist.index[user_dist >= 5])]

# Update movie and user distribution

movie_dist = data[["movieId"]].groupby("movieId",as_index=False).size()
user_dist = data[["userId"]].groupby("userId",as_index=False).size()



(20720, 136677)

In [0]:
import numpy as np
user_list = user_dist.index

In [0]:
#user_list

user_dist=user_dist.sample(frac=1)
user_list = user_dist.index



Straight Forward Splitting

In [0]:
user_list_tr = user_list[:-20000]

user_list_te=user_list[-20000:-10000]
user_list_vd = user_list[-10000:]


Done split for the moment

In [0]:
data_tr = data[data['userId'].isin(user_list_tr)]


In [0]:
unique_movie = pd.unique(data_tr['movieId'])


In [0]:
movie2id = dict((sid,i) for (i, sid) in enumerate(unique_movie))
user2id = dict((sid,i) for (i,sid) in enumerate(user_list))
#movie2id has movies only in the traning dataset whereas user2id has all the users

In [0]:
#save the lists
import os
data_root = '/Root folder to save the data/'
save_dir = data_root+ 'lists'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)
with open(os.path.join(save_dir, 'unique_movie.txt'), 'w') as f:
    for movie in unique_movie:
        f.write('%s\n' % movie)
#for movie in unique_movie:
#  print (movie)

saved movie list

In [0]:
def numerize(dt):
    uid = list(map(lambda x: user2id[x], dt['userId']))
    sid = list(map(lambda x: movie2id[x], dt['movieId']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])
#train_data = numerize(data_tr)


In [0]:
train_data = numerize(data_tr)
train_data.to_csv(os.path.join(save_dir, 'train.csv'), index=False)


the train_data only contain the mapping of the users and movies where the actual user and movies are in the user2id, movie2id lists


Only val split atm

ATM only develop single model :
denoising autoincoders with multinomial likelihood fn
Why: Cause the time dude

In [0]:
#load data and train

#unique_movie
from scipy import sparse

num_item = len(unique_movie)
num_item
len(user_list_tr)
n_users=data_tr['userId'].max() + 1
#data_tr['userId'].head()
rows, cols = train_data['uid'], train_data['sid']
training_data = sparse.csr_matrix((np.ones_like(rows),(rows, cols)), dtype='float64',shape=(n_users, num_item))
# data has entry 1 where the user has rated the movie: highly sparse matrix


In [0]:
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [0]:
data_test = data.loc[data['userId'].isin(user_list_te)]
data_test = data_test.loc[data['movieId'].isin(unique_movie)]
data_test_tr,data_test_te = split_train_test_proportion(data_test)

In [0]:
data_test_tr = numerize(data_test_tr)
data_test_te = numerize(data_test_te)

In [0]:
data_test_tr.to_csv(os.path.join(save_dir, 'data_test_tr.csv'), index=False)
data_test_te.to_csv(os.path.join(save_dir, 'data_test_te.csv'), index=False)


In [0]:
data_vd = data.loc[data['userId'].isin(user_list_vd)]
data_vd = data_vd.loc[data['movieId'].isin(unique_movie)]
data_vd_tr,data_vd_te = split_train_test_proportion(data_vd)

In [0]:
data_vd_tr = numerize(data_vd_tr)
data_vd_te = numerize(data_vd_te)

In [0]:
data_vd_tr.to_csv(os.path.join(save_dir, 'data_vd_tr.csv'), index=False)
data_vd_te.to_csv(os.path.join(save_dir, 'data_vd_te.csv'), index=False)

In [0]:
# Model in pytorch
## l2 regularizers are included by default in optimizers??
##will defining the model tf way later
import torch
import torch.nn as nn
import torch.nn.parallel # not using multi-gpu for the time being
from torch.autograd import Variable


class model_da(nn.Module):
    def __init__(self,in_feature,latent_feature,weight_deca=0.0):
      super(model_da, self).__init__()
      self.weight_decay = weight_deca
      self.linear1 = nn.Linear(in_feature, latent_feature, bias=True)
      self.linear2 = nn.Linear(latent_feature, in_feature, bias=True)
      self.tanh = nn.Tanh()
    def forward(self,inp,drop_prob=0):
      x1 = torch.nn.functional.normalize(inp,p=2,dim=1) #default 2nd degree normalizer
      x2 = torch.nn.Dropout(p=drop_prob)(x1)
      x_lat = self.tanh(self.linear1(x2))

      x_out = self.linear2(x_lat)

      return x_out
    def loss(self,pred,gt):
      pred = nn.functional.log_softmax(pred,dim=-1)
      loss_mat = pred*gt
      loss = -loss_mat.mean(1)
      return loss.mean(0).view(1)
    def l2_reg(self):
        l2_reg = Variable(torch.FloatTensor(1), requires_grad=True)
        
        for k, m in self.state_dict().items():
            if k.endswith('.weight'):
                l2_reg = l2_reg + torch.norm(m, p=2) ** 2
            l2_reg = self.weight_decay * l2_reg
        #if self.cuda2:
        #    l2_reg = l2_reg.cuda()
        return l2_reg[0]

In [0]:
training_data = training_data
N = training_data.shape[0]
idxlist = range(N)
batch_size = 500
batches_per_epoch = int(np.ceil(float(N) / batch_size))


In [0]:
def load_tr_te_data(tp_tr, tp_te):
    #tp_tr = pd.read_csv(csv_file_tr)
    #tp_te = pd.read_csv(csv_file_te)


    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, num_item))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, num_item))
    return data_tr, data_te

In [0]:
vad_data_tr, vad_data_te = load_tr_te_data(data_vd_tr,data_vd_te)
                              

In [0]:
import bottleneck as bn
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [0]:
# weight initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data,std=0.001)
        m.bias.data.clamp_(-2*0.001, 2*0.001) # temporary fix: this will set the values to 0 but 
        # in tf the values are redrawn : write a function for this if necessary

In [0]:
#call model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model_da(num_item,200,weight_deca=0.01 /500)
model.apply(weights_init)

model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0)
inputs = torch.FloatTensor(batch_size,num_item )
out_gr = torch.FloatTensor(batch_size,num_item )

inputs.to(device)
out_gr.to(device)


In [0]:
#define default pytorch dataloader
from torch.utils import data
from scipy import sparse
# assumes 'training_data' is np array
# need to chage the structure of array from [a,b] to [a,[b,1]]
#training_data= sparse.csr_matrix.toarray(training_data)
dataloader= torch.utils.data.DataLoader(training_data,batch_size = 500,drop_last=True)


In [0]:
#for validation data
batch_size_vd = batch_size
no_batches = 10000/batch_size_vd
batch_list_vd = np.arange((no_batches)).astype('int32')
rng = np.random.default_rng()
rng.shuffle(batch_list_vd)


In [0]:
# loop for training
training_data
138494/500
batches_per_epoch
batch_list = np.arange((batches_per_epoch-2))
rng = np.random.default_rng()
#batch_list = np.array(batch_list)
#list(range(batch_list))
#print (batch_list)
rng.shuffle(batch_list)
#print (batch_list)
ndcg_vad =[]
from torch.autograd import Variable
ndcgs_vad=[]
for epoch in range(200):
  #data_iter = iter(dataloader)
  
  rng.shuffle(batch_list)
  #print (batch_list)
  i = 0
  while i < len(batch_list):
    #data = data_iter.next()
    data = training_data[batch_list[i]*batch_size:(batch_list[i]+1)*batch_size].toarray()
    
    i+=1
    
    model.zero_grad()
    inputs=data
    inputv = Variable(torch.Tensor(inputs)).to(device)
    #print (inputv.shape)
    pred = model(inputv)
    los = model.loss(pred,inputv) 
    los_reg = model.l2_reg().to(device)
    loss_total = los+los_reg
    loss_total.backward()
    optimizer.step()
  ndcg_dist = []
  j=0
  while j< len(batch_list_vd):
    
    
    data_vd_tr = vad_data_tr[batch_list_vd[j]*batch_size:(batch_list_vd[j]+1)*batch_size].toarray()
    data_vd_te = vad_data_te[batch_list_vd[j]*batch_size:(batch_list_vd[j]+1)*batch_size]#.toarray()
    with torch.no_grad():
      pred_val = model(Variable(Variable(torch.Tensor(data_vd_tr)).to(device)))
      pred_val = pred_val.cpu().detach().numpy()
      pred_val[data_vd_tr.nonzero()] = -np.inf
      ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, data_vd_te))
    j+=1
  ndcg_dist = np.concatenate(ndcg_dist)
  ndcg_ = ndcg_dist.mean()
  #print (ndcg_)
  ndcg_vad.append(ndcg_)

  



In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(1,201,1)

plt.figure(figsize=(12, 3))
plt.plot(x,ndcgs_vad,'r-',label='final')
plt.ylabel("Validation NDCG@100")
plt.xlabel("Epochs")
plt.legend()
plt.savefig(save_dir+'/combine')